In [ ]:
from nuplan.common.maps.nuplan_map.map_factory import get_maps_api, get_maps_db

MAP_LOCATIONS = {"sg-one-north", "us-ma-boston", "us-nv-las-vegas-strip", "us-pa-pittsburgh-hazelwood"}

In [ ]:
length = {0: 1, 1: 2, 2: 3, 3: 4, 4: 0}
max(length, key=length.get)

In [ ]:
import geopandas as gpd
import pyogrio


# Path to your .gpkg file
# path_to_gpkg = "/home/daniel/nuplan/dataset/maps/us-nv-las-vegas-strip/9.15.1915/map.gpkg"
path_to_gpkg = "/home/daniel/nuplan/dataset/maps/us-ma-boston/9.12.1817/map.gpkg"
# path_to_gpkg = "/home/daniel/nuplan/dataset/maps/us-pa-pittsburgh-hazelwood/9.17.1937/map.gpkg"
# path_to_gpkg = "/home/daniel/nuplan/dataset/maps/sg-one-north/9.17.1964/map.gpkg"


# List all available layers
layers = [
    "baseline_paths",
    "carpark_areas",
    "generic_drivable_areas",
    "dubins_nodes",
    "lane_connectors",
    "intersections",
    "boundaries",
    "crosswalks",
    "lanes_polygons",
    "lane_group_connectors",
    "lane_groups_polygons",
    "walkways",
    "gen_lane_connectors_scaled_width_polygons",
    "meta",
]


def load_layer(layer_name: str) -> gpd.geodataframe:

    map_meta = gpd.read_file(path_to_gpkg, layer="meta", engine="pyogrio")
    projection_system = map_meta[map_meta["key"] == "projectedCoordSystem"]["value"].iloc[0]

    gdf_in_pixel_coords = pyogrio.read_dataframe(path_to_gpkg, layer=layer_name, fid_as_index=True)
    gdf_in_utm_coords = gdf_in_pixel_coords.to_crs(projection_system)

    # For backwards compatibility, cast the index to string datatype.
    #   and mirror it to the "fid" column.
    gdf_in_utm_coords.index = gdf_in_utm_coords.index.map(str)
    gdf_in_utm_coords["fid"] = gdf_in_utm_coords.index

    return gdf_in_utm_coords


pyogrio.read_dataframe(path_to_gpkg)

In [ ]:
import pandas as pd


layers = [
    "baseline_paths",
    "carpark_areas",
    "generic_drivable_areas",
    "dubins_nodes",
    "lane_connectors",
    "intersections",
    "boundaries",
    "crosswalks",
    "lanes_polygons",
    "lane_group_connectors",
    "lane_groups_polygons",
    "road_segments",
    "stop_polygons",
    "traffic_lights",
    "walkways",
    "gen_lane_connectors_scaled_width_polygons",
    "meta",
]


def non_nan_set(series: pd.Series) -> set:
    return set(series.dropna().values)


lanes = load_layer("intersections")
boundaries = load_layer("boundaries")


def get_lane(id: int):
    return lanes[lanes["fid"] == str(id)]


def get_right_boundary(lane):
    return boundaries[boundaries["fid"] == str(lane["right_boundary_fid"].iloc[0])]


def get_left_boundary(lane):
    return boundaries[boundaries["fid"] == str(lane["left_boundary_fid"].iloc[0])]


# boundaries

# correct_lane_fid = 47716
# error_lane_fid = 48508

# correct_lane_df = get_lane(correct_lane_fid)
# error_lane_df = get_lane(error_lane_fid)


# 46552,46553
# test_id = 46553

print(lanes.keys())
non_nan_set(lanes.intersection_type_fid)



In [ ]:
baseline_paths = load_layer("baseline_paths")


import matplotlib.pyplot as plt
from networkx import center

# Create a figure
fig, ax = plt.subplots(figsize=(15, 12))

# Get unique path_type_fid values
path_types = baseline_paths["path_type_fid"].unique()

# Define colors for different path types
colors = ["blue", "red", "green", "brown", "purple", "orange", "pink"]

# Plot each path type with a different color
target_label = 0
for i, path_type in enumerate(path_types):
    # Filter baseline_paths by path_type
    paths_of_type = baseline_paths[baseline_paths["path_type_fid"] == path_type]

    if path_type == target_label:
        alpha = 1.0
        target_fids = paths_of_type.fid.tolist()
    else:
        alpha = 0.5

    # Plot these paths with a specific color
    paths_of_type.plot(ax=ax, color=colors[i % len(colors)], label=f"Path Type {path_type}", alpha=alpha)

# Add title and legend
ax.set_title("Baseline Paths by Path Type", fontsize=16)
ax.legend()
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.axis("equal")  # Maintain aspect ratio

center_zoom = False  # Set to True to zoom into the center of the data
fid_zoom = target_fids[25]

if center_zoom:
    # Get the bounds of the data for better focusing
    x_min, y_min, x_max, y_max = baseline_paths.total_bounds

    # Calculate center coordinates
    center_x = (x_min + x_max) / 2
    center_y = (y_min + y_max) / 2

    # Set axis limits to zoom into the center (using 30% of the total range)
    range_x = x_max - x_min
    range_y = y_max - y_min
    zoom_factor = 2.0

    ax.set_xlim(center_x - range_x * zoom_factor / 2, center_x + range_x * zoom_factor / 2)
    ax.set_ylim(center_y - range_y * zoom_factor / 2, center_y + range_y * zoom_factor / 2)

elif fid_zoom:
    # Filter to get the specific element with the given FID
    specific_path = baseline_paths[baseline_paths["fid"] == str(fid_zoom)]

    if not specific_path.empty:
        # Get the bounds of the specific path
        x_min, y_min, x_max, y_max = specific_path.total_bounds

        # Add some padding around the element
        padding = 100  # meters
        ax.set_xlim(x_min - padding, x_max + padding)
        ax.set_ylim(y_min - padding, y_max + padding)

        # Highlight the specific element
        # specific_path.plot(ax=ax, color="yellow", linewidth=3, zorder=10)

        # Update title to show we're zoomed to a specific FID
        ax.set_title(f"Baseline Path - FID {fid_zoom}", fontsize=16)
    else:
        print(f"FID {fid_zoom} not found in baseline_paths")


# Add a title to indicate we're looking at the center
ax.set_title("Baseline Paths by Path Type (Center Zoom)", fontsize=16)


# Convert the specific path coordinates to WGS84 (latitude/longitude) for Google Maps
if fid_zoom and not specific_path.empty:
    # Create a copy to avoid modifying the original
    wgs84_path = specific_path.copy()

    # Convert from the current projection to WGS84 (EPSG:4326)
    wgs84_path = wgs84_path.to_crs("EPSG:4326")

    # Get the centroid of the path for easier lookup
    centroid = wgs84_path.geometry.iloc[0].centroid

    # Display the coordinates
    print(f"\nGoogle Maps coordinates for FID {fid_zoom}:")
    print(f"Latitude: {centroid.y}, Longitude: {centroid.x}")
    print(f"Google Maps link: https://www.google.com/maps?q={centroid.y},{centroid.x}")

    # Add a text annotation showing coordinates on the plot
    ax.annotate(
        f"Lat: {centroid.y:.6f}, Lon: {centroid.x:.6f}",
        xy=(0.05, 0.05),
        xycoords="axes fraction",
        bbox=dict(boxstyle="round,pad=0.5", fc="white", alpha=0.8),
        fontsize=10,
    )
plt.tight_layout()
plt.show()

In [ ]:
baseline_paths = load_layer("baseline_paths")
intersections = load_layer("intersections")  # Load the intersections layer

import matplotlib.pyplot as plt
from networkx import center

# Create a figure
fig, ax = plt.subplots(figsize=(15, 12))

# Get unique intersection types
intersection_types = intersections["intersection_type_fid"].unique()

# Define colors for different intersection types
colors = ["blue", "red", "green", "brown", "purple", "orange", "pink"]

# Plot each intersection type with a different color
target_label = 2  # Target intersection type to highlight
for i, intersection_type in enumerate(intersection_types):
    # Filter intersections by type
    intersections_of_type = intersections[intersections["intersection_type_fid"] == intersection_type]

    if intersection_type == target_label:
        alpha = 1.0
        target_fids = intersections_of_type.fid.tolist()
    else:
        alpha = 0.5

    # Plot these intersections with a specific color
    intersections_of_type.plot(ax=ax, color=colors[i % len(colors)], 
                            label=f"Intersection Type {intersection_type}", alpha=alpha)

# Add title and legend
ax.set_title("Intersections by Type", fontsize=16)

ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.axis("equal")  # Maintain aspect ratio

center_zoom = False  # Set to True to zoom into the center of the data
fid_zoom = target_fids[0] if 'target_fids' in locals() and len(target_fids) > 0 else None

if center_zoom:
    # Get the bounds of the data for better focusing
    x_min, y_min, x_max, y_max = intersections.total_bounds

    # Calculate center coordinates
    center_x = (x_min + x_max) / 2
    center_y = (y_min + y_max) / 2

    # Set axis limits to zoom into the center (using 30% of the total range)
    range_x = x_max - x_min
    range_y = y_max - y_min
    zoom_factor = 2.0

    ax.set_xlim(center_x - range_x * zoom_factor / 2, center_x + range_x * zoom_factor / 2)
    ax.set_ylim(center_y - range_y * zoom_factor / 2, center_y + range_y * zoom_factor / 2)

elif fid_zoom:
    # Filter to get the specific intersection with the given FID
    specific_intersection = intersections[intersections["fid"] == fid_zoom]

    if not specific_intersection.empty:
        # Get the bounds of the specific intersection
        x_min, y_min, x_max, y_max = specific_intersection.total_bounds

        # Add some padding around the element
        padding = 100  # meters
        ax.set_xlim(x_min - padding, x_max + padding)
        ax.set_ylim(y_min - padding, y_max + padding)

        # Update title to show we're zoomed to a specific FID
        ax.set_title(f"Intersection - FID {fid_zoom} (Type {specific_intersection['intersection_type_fid'].iloc[0]})", fontsize=16)
    else:
        print(f"FID {fid_zoom} not found in intersections")


# Add a title if not zooming to a specific FID
if not fid_zoom:
    ax.set_title("Intersections by Type", fontsize=16)

# Convert the specific intersection coordinates to WGS84 (latitude/longitude) for Google Maps
if fid_zoom and 'specific_intersection' in locals() and not specific_intersection.empty:
    # Create a copy to avoid modifying the original
    wgs84_intersection = specific_intersection.copy()

    # Convert from the current projection to WGS84 (EPSG:4326)
    wgs84_intersection = wgs84_intersection.to_crs("EPSG:4326")

    # Get the centroid of the intersection for easier lookup
    centroid = wgs84_intersection.geometry.iloc[0].centroid

    # Display the coordinates
    print(f"\nGoogle Maps coordinates for Intersection FID {fid_zoom}:")
    print(f"Latitude: {centroid.y}, Longitude: {centroid.x}")
    print(f"Google Maps link: https://www.google.com/maps?q={centroid.y},{centroid.x}")

    # Add a text annotation showing coordinates on the plot
    ax.annotate(
        f"Lat: {centroid.y:.6f}, Lon: {centroid.x:.6f}",
        xy=(0.05, 0.05),
        xycoords="axes fraction",
        bbox=dict(boxstyle="round,pad=0.5", fc="white", alpha=0.8),
        fontsize=10,
    )
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
def _get_set(df, column: str):
    """
    Returns a set of unique values in the specified column of the DataFrame.
    """
    return set(df[column].unique())

In [ ]:
_get_set(lanes, "lane_index")

In [ ]:
lanes[lanes["left_boundary_fid"] == test_id].keys()

In [ ]:
get_left_boundary(correct_lane_df)

In [ ]:
get_right_boundary(correct_lane_df)

In [ ]:
get_left_boundary(error_lane_df)

In [ ]:
get_right_boundary(error_lane_df)

In [ ]:
import pandas as pd

lane_df = load_layer("generic_drivable_areas")
lane_df

In [ ]:
walkways_df = load_layer("carparks")
walkways_df

In [ ]:
gen_lane_connectors_scaled_width_polygons_df = load_layer("boundaries")

gen_lane_connectors_scaled_width_polygons_df[gen_lane_connectors_scaled_width_polygons_df["fid"] == "17950"]

In [ ]:
[None] * 10

In [ ]:
import pandas as pd

lane_df = load_layer("baseline_paths")
lane_df

In [ ]:
def mps_to_mph(mph: float) -> float:
    """
    Convert meters per second to miles per hour.
    :param mph: miles per hour [mi/h]
    :return: meters per second [m/s]
    """
    return mph / 0.44704


mps_to_mph(6.705409029950827)

In [ ]:
# fix, ax = plt.subplots(figsize=(10, 10))

# type = 3

# for i in np.random.choice(len(geoms[type]), 10):
#     ax.plot(*geoms[type][i].coords.xy, color="blue")

In [ ]:
geoms[2][1]

In [ ]:
geoms[3][1]